# Canadian Housing Market Forecasting: Time Series Analysis & Modelling.
Description:  

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Data Ingestion and Data Cleaning
Description: 

In [2]:
#csv for mortgage rates chartered_bank_interest.csv
#ingest the csv 
#take all dates after 1975-01-01
#take only the column named 5-year Conventional mortgage
df_chartered_bank_i = pd.read_csv('../Resources/Datasets/chartered_bank_interest.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df_chartered_bank_i.head()

,Prime rate,1-year Conventional mortgage,3-year Conventional mortgage,5-year Conventional mortgage,1-year GIC,3-year GIC,5-year GIC,5-year personal fixed term,"Daily Interest Savings (balances over $100,000)",Non-Chequable Savings Deposits
date,,,,,,,,,,
1967-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5
1967-04-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5
1967-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5
1967-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5
1967-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5


In [3]:
#df_chartered_bank_i.index = df_chartered_bank_i.index.strftime('%d/%m/%Y')

In [4]:
#df_chartered_bank_i.tail()

In [5]:
# Convert daily data to monthly through the resample function
df_chartered_bank_i.index = pd.to_datetime(df_chartered_bank_i.index)
df_chartered_bank_i.resample('1Q').mean().tail()

,Prime rate,1-year Conventional mortgage,3-year Conventional mortgage,5-year Conventional mortgage,1-year GIC,3-year GIC,5-year GIC,5-year personal fixed term,"Daily Interest Savings (balances over $100,000)",Non-Chequable Savings Deposits
date,,,,,,,,,,
2020-12-31,2.45,3.090000,3.556154,4.79,0.350000,0.600000,0.857692,0.700000,0.01,0.01
2021-03-31,2.45,2.813077,3.490000,4.79,0.315385,0.530769,0.853846,0.700000,0.01,0.01
2021-06-30,2.45,2.790000,3.490000,4.79,0.250000,0.553846,1.003846,0.700000,0.01,0.01
2021-09-30,2.45,2.790000,3.490000,4.79,0.250000,0.600000,1.000000,0.700000,0.01,0.01
2021-12-31,2.45,2.790000,3.490000,4.79,0.261111,0.616667,1.020000,0.764444,0.01,0.01


In [22]:
# Data ingestion - reading the CSV in from Statistics Canada and the Bank of Canada
# Credit liabilities of households.csv
# Residential mortgages 9+
df_Residential_mortgages = pd.read_csv('../Resources/Datasets/Credit liabilities of households date.csv')
df_Residential_mortgages.head()

,Date,Non-mortgage loans,Chartered banks 1,Personal loans 1 2,Of which: Auto loans 1 2,Credit cards 1 2,Lines of credit 1 2 4,"Lines of credit, of which: excluding home equity lines of credit 1 2",Other personal loans 1 2,Unincorporated business 1 2,...,Of which: real estate secured lending 14,Residential mortgages 14,Chartered banks 10 14,Non-banks 10 14,Non-residential mortgages 14,Chartered banks 13 14,Non-banks 13 14,Home equity lines of credit 14,Chartered banks 14,Non-banks 14 15
0,2000-04-01,"229,609","133,065","40,604",0,"15,993","33,269","7,070","26,650","16,549",...,"432,794","403,745","247,468","156,277","2,850",686,"2,164","26,199","26,199",0
1,2001-04-01,"248,485","143,748","39,886",0,"19,691","42,489","9,649","25,075","16,607",...,"451,675","415,700","256,455","159,245","3,016",681,"2,335","32,959","32,841",118
2,2002-04-01,"265,948","153,523","39,703",0,"24,053","50,757","12,293","22,225","16,785",...,"487,095","445,002","284,314","160,688","3,122",683,"2,439","38,971","38,465",506
3,2003-04-01,"287,077","169,919","38,887",0,"31,027","63,488","15,577","19,932","16,585",...,"529,669","477,469","301,420","176,049","3,342",717,"2,625","48,858","47,911",947
4,2004-04-01,"317,766","187,594","39,442",0,"33,139","80,496","19,027","18,590","15,927",...,"586,504","520,265","316,898","203,367","3,713",787,"2,926","62,526","61,469","1,057"


In [23]:
#df_Residential_mortgages.index = pd.to_datetime(df_Residential_mortgages.index)
#df_Residential_mortgages.resample('1Q').mean().tail()

In [24]:
df_Residential_mortgages = df_Residential_mortgages.sort_values(by = 'Date')
df_Residential_mortgages.head()

,Date,Non-mortgage loans,Chartered banks 1,Personal loans 1 2,Of which: Auto loans 1 2,Credit cards 1 2,Lines of credit 1 2 4,"Lines of credit, of which: excluding home equity lines of credit 1 2",Other personal loans 1 2,Unincorporated business 1 2,...,Of which: real estate secured lending 14,Residential mortgages 14,Chartered banks 10 14,Non-banks 10 14,Non-residential mortgages 14,Chartered banks 13 14,Non-banks 13 14,Home equity lines of credit 14,Chartered banks 14,Non-banks 14 15
149,1990-01-01,"121,607","77,976","31,781",0,"9,090","5,550","1,264","16,523","15,032",...,"235,302","228,663","85,704","142,959","2,353",283,"2,070","4,286","4,286",0
117,1990-02-01,"121,404","77,819","31,769",0,"8,768","5,695","1,297","16,585","15,002",...,"236,297","229,555","86,873","142,682","2,344",283,"2,061","4,398","4,398",0
245,1990-03-01,"122,358","78,386","31,878",0,"8,728","5,841","1,330","16,828","15,111",...,"238,489","231,604","87,830","143,774","2,374",289,"2,085","4,511","4,511",0
22,1990-04-01,"122,922","78,566","32,099",0,"8,883","5,989","1,383","16,630","14,965",...,"241,170","234,168","89,060","145,108","2,396",296,"2,100","4,606","4,606",0
277,1990-05-01,"123,435","78,941","32,316",0,"9,050","6,022","1,409","16,699","14,854",...,"243,796","236,766","90,473","146,293","2,418",300,"2,118","4,612","4,612",0


In [25]:
#transpose 
#res_mortgage_transp = df_Residential_mortgages.T
#res_mortgage_transp.head()
df_Residential_mortgages = df_Residential_mortgages.set_index('Date')
df_Residential_mortgages.head()

,Non-mortgage loans,Chartered banks 1,Personal loans 1 2,Of which: Auto loans 1 2,Credit cards 1 2,Lines of credit 1 2 4,"Lines of credit, of which: excluding home equity lines of credit 1 2",Other personal loans 1 2,Unincorporated business 1 2,Non-banks 1,...,Of which: real estate secured lending 14,Residential mortgages 14,Chartered banks 10 14,Non-banks 10 14,Non-residential mortgages 14,Chartered banks 13 14,Non-banks 13 14,Home equity lines of credit 14,Chartered banks 14,Non-banks 14 15
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-01,"121,607","77,976","31,781",0,"9,090","5,550","1,264","16,523","15,032","43,631",...,"235,302","228,663","85,704","142,959","2,353",283,"2,070","4,286","4,286",0
1990-02-01,"121,404","77,819","31,769",0,"8,768","5,695","1,297","16,585","15,002","43,585",...,"236,297","229,555","86,873","142,682","2,344",283,"2,061","4,398","4,398",0
1990-03-01,"122,358","78,386","31,878",0,"8,728","5,841","1,330","16,828","15,111","43,972",...,"238,489","231,604","87,830","143,774","2,374",289,"2,085","4,511","4,511",0
1990-04-01,"122,922","78,566","32,099",0,"8,883","5,989","1,383","16,630","14,965","44,356",...,"241,170","234,168","89,060","145,108","2,396",296,"2,100","4,606","4,606",0
1990-05-01,"123,435","78,941","32,316",0,"9,050","6,022","1,409","16,699","14,854","44,494",...,"243,796","236,766","90,473","146,293","2,418",300,"2,118","4,612","4,612",0


In [26]:
#df_Residential_mortgages = df_Residential_mortgages.sort_index().truncate(before='Residential mortgages 14', after='Residential mortgages 14', axis=1)

In [27]:
# Convert monthly data to quarterly through the resample function
#df_Residential_mortgages.index = pd.to_datetime(df_Residential_mortgages.index)
#df_Residential_mortgages.resample('1Q').mean().tail()

In [28]:
# Drop unwanted columns
df_Residential_mortgages = df_Residential_mortgages[['Residential mortgages 14']]
df_Residential_mortgages.head()

,Residential mortgages 14
Date,
1990-01-01,"228,663"
1990-02-01,"229,555"
1990-03-01,"231,604"
1990-04-01,"234,168"
1990-05-01,"236,766"


In [12]:
# Reset index and change headers
#res_mortgage_transp = res_mortgage_transp.reset_index(drop=False)
#new_header = res_mortgage_transp.iloc[0] #grab the first row for the header
#res_mortgage_transp = res_mortgage_transp[1:] #take the data less the header row
#res_mortgage_transp.columns = new_header #set the header row as the dataframe header
#res_mortgage_transp.head()

In [13]:
#res_mortgage_transp.dtypes

In [14]:
#possible_formats = ['%b-y%', '%y-%b']
#res_mortgage_transp['Date'] = pd.to_datetime(res_mortgage_transp['Date'], errors='coerce', format=fmt) for fmt in possible_formats
#res_mortgage_transp['Date'] = res_mortgage_transp['Date'].dt.strftime('%m/%y')
#res_mortgage_transp['Date'].tail()
#start_date = '01-Jan'
#end_date = '21-Sep'

#res_mortgage_transp_mask_1 = (res_mortgage_transp['Date'] > start_date) & (res_mortgage_transp['Date'] <= end_date)
#res_mortgage_transp_mask_1.head()

In [29]:
# Reset date column as index and rename main column 
#df_Residential_mortgages = df_Residential_mortgages.set_index('Date')
df_Residential_mortgages = df_Residential_mortgages.rename(columns={'Residential mortgages 14': 'Residential Mortgages'})
df_Residential_mortgages.head()

,Residential Mortgages
Date,
1990-01-01,"228,663"
1990-02-01,"229,555"
1990-03-01,"231,604"
1990-04-01,"234,168"
1990-05-01,"236,766"


In [38]:
# Convert daily data to monthly through the resample function
df_Residential_mortgages.index = pd.to_datetime(df_Residential_mortgages.index)
df_Residential_mortgages_index = df_Residential_mortgages.resample('1Q').mean().tail()
#df_Residential_mortgages.set_index(df_Residential_mortgages_index, inplace=True) 
df_Residential_mortgages['date'] = df_Residential_mortgages_index
df_Residential_mortgages.tail()

#shopify_csv = shopify_csv.set_index(pd.to_datetime(shopify_csv["date"], infer_datetime_format=True))


#res_mortgage_transp['Date'] = pd.to_datetime(res_mortgage_transp['Date'], format='%b-%y').add(pd.offsets.MonthEnd(0))
#res_mortgage_transp['Date'] = pd.to_datetime(res_mortgage_transp['Date'])
#df['end_date'] = pd.to_datetime(
#    df['end_date'], format='%b-%y'
#).add(pd.offsets.MonthEnd(0))

ValueError: Wrong number of items passed 0, placement implies 1

In [39]:
df_Residential_mortgages_index.dtypes

Series([], dtype: object)

In [17]:
#take the csv in 
#extract this column name: Credit liabilities of households 1 2 3
#extract this column name: Residential mortgages 9 
#flip the row into a column (transpose method)
#since the dates won't match, we need to add null values


In [18]:
# House afforability index 
df_House_afford_index  = pd.read_csv('../Resources/Datasets/AffordabilityIndex.csv')
df_House_afford_index.head()


,date,New housing price index,Housing Affordability Index
0,1980Q3,NaN,0.420
1,1980Q4,NaN,0.461
2,1981Q1,NaN,0.482
3,1981Q2,NaN,0.534
4,1981Q3,NaN,0.626
